# **PORTFOLIOS LP**

By Marta González Pérez

## **CONTEXT** ##

The goal of this problem is to allocate $100,000 among three financial assets: stocks, bonds and a savings account, in order to maximize expected return. The allocation must satisfy risk and diversification constraints:
- No more than 50% of the total investment in stocks.

- At least 20% in the savings account.

- All investments must be non-negative.

## Linear Programming Formulation

The problem is modeled using linear programming, as follows:

- Decision variables:

  - $x_1$ = amount invested in stocks

  - $x_2$ = amount invested in bonds

  - $x_3$ = amount invested in the savings account
  

- Objective function:

  Maximize total expected return:
  
  $$
  Z = 0.08 x_1 + 0.05 x_2 + 0.02 x_3
  $$


- Constraints:
  - $x_1 + x_2 + x_3 \leq 100,000$ (Total budget)

  - $x_1 \leq 0.5(x_1 + x_2 + x_3)$ (Max 50% in stocks)

  - $x_3 \geq 0.2(x_1 + x_2 + x_3)$ (Min 20% in savings)
  
  - $x_1, x_2, x_3 \geq 0$ (Non-negativity)


- Results:

  - Stocks: $50,000$

  - Bonds: $30,000$

  - Savings: $20,000$

  - Maximum expected return: $5,900$


The solution satisfies all constraints, respecting both the diversification and minimum savings requirements.


## Code

- Import PuLP library to build and solve the optimization problem.
- Define the problem as a maximization model.
- Create decision variables representing the investment amounts in stocks, bonds, and savings, all restricted to be non-negative.
- Add the objective function to maximize the total return based on the given interest rates.
- Add constraints to:
    - Limit total investment to $100,000
    - Restrict stocks to at most 50% of total funds
    - Ensure at least 20% is placed in savings
    - Enforce non-negativity of all variables
- Run the solver (CBC) to find the optimal solution.
- Print results, showing the optimal investment allocation and the computed maximum return.

In [44]:
#EXERCISE LINEAR PROGRAMMING

from pulp import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


prob = LpProblem("Max_return_portfolios", LpMaximize)

x1 = LpVariable("Stocks", lowBound = 0)
x2 = LpVariable("Bonds", lowBound = 0)
x3 = LpVariable("Savings", lowBound = 0)
y = LpVariable("Return", lowBound = 0)

prob += 0.08*x1 + 0.05*x2 + 0.02*x3, "Max benefit"
prob += x1 + x2 + x3 <= 100000, "Total budget"
prob += x1 <= 0.5*(x1+x2+x3), "Max 50 stocks"
prob += x3 >= 0.2*(x1+x2+x3), "Min 20 savings"
prob += x1 >= 0, "x1 higher 0"
prob += x2 >= 0, "x2 higher 0"
prob += x3 >= 0, "x3 higher 0"

prob.solve(PULP_CBC_CMD(msg=False))

print("State of the solution", LpStatus[prob.status])
print(f"x1 (Stocks): {x1.varValue}")
print(f"x2 (Bonds): {x2.varValue}")
print(f"x3 (Savings): {x3.varValue}")
print(f"Total max benefit: {value(prob.objective)}$\n")

State of the solution Optimal
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 5900.0$



### Interpretation of the Results

The solution found is optimal, meaning it gives the best possible result under the given constraints.

- **x1 (Stocks):** \$50,000 invested in stocks  
- **x2 (Bonds):** \$30,000 invested in bonds  
- **x3 (Savings):** \$20,000 kept in savings  

The total benefit from this investment plan is \$5,900.


## Sensitivity Analysis

To understand how the optimal portfolio reacts to changes in constraints and returns, we performed sensitivity analysis.

### Shadow Prices

| Constraint | Shadow Price | Interpretation |
|-----------|--------------|----------------|
| Total budget | +0.059 | Increasing available capital increases total return. |
| Max % in stocks | +0.030 | Allowing a higher stock proportion increases the return. |
| Min % in savings | −0.030 | Forcing more savings reduces the return (savings have low yield). |

- Positive shadow price → relaxing the constraint increases the objective.  
- Negative shadow price → tightening the constraint reduces the objective.

In [45]:
#SENSIBILITY ANALYSIS: shadow prices

for name, c in list(prob.constraints.items()):
    print({"Name":name, "Shadow price":c.pi, "Slack": c.slack})

#If we increase the total budget or the percentage of maximum stock, it gives us benefit (positive gain).
#However, if we increase the percentage of minimum savings, it takes away our benefit (negative gain).

{'Name': 'Total_budget', 'Shadow price': 0.059, 'Slack': -0.0}
{'Name': 'Max_50_stocks', 'Shadow price': 0.03, 'Slack': -0.0}
{'Name': 'Min_20_savings', 'Shadow price': -0.03, 'Slack': -0.0}
{'Name': 'x1_higher_0', 'Shadow price': -0.0, 'Slack': -50000.0}
{'Name': 'x2_higher_0', 'Shadow price': -0.0, 'Slack': -30000.0}
{'Name': 'x3_higher_0', 'Shadow price': -0.0, 'Slack': -20000.0}



### Effect of Changing Constraints

#### Maximum % in Stocks (x1 ≤ n(x1 + x2 + x3))
As the stock limit increases (20% → 80%), more investment goes into stocks and the total return rises:

| Max % in Stocks | Stocks | Bonds | Savings | Return |
|----|----|----|----|----|
| 20% | 20k | 60k | 20k | $5000 |
| 50% | 50k | 30k | 20k | $5900 |
| 80% | 80k | 0 | 20k | $6800 |

**Higher stock allowance = higher return.**


In [24]:
#SENSIBILITY ANALYSIS: x1 <= n*(x1+x2+x3)

for n in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    prob = LpProblem("Max_return_portfolios", LpMaximize)

    x1 = LpVariable("Stocks", lowBound = 0)
    x2 = LpVariable("Bonds", lowBound = 0)
    x3 = LpVariable("Savings", lowBound = 0)
    y = LpVariable("Return", lowBound = 0)

    prob += 0.08*x1 + 0.05*x2 + 0.02*x3, "Max benefit"
    prob += x1 + x2 + x3 <= 100000, "Total budget"
    prob += x1 <= n*(x1+x2+x3), "Max n stocks"
    prob += x3 >= 0.2*(x1+x2+x3), "Min 20 savings"
    prob += x1 >= 0, "x1 higher 0"
    prob += x2 >= 0, "x2 higher 0"
    prob += x3 >= 0, "x3 higher 0"

    if x1 <= n*(x1+x2+x3):
        prob.solve(PULP_CBC_CMD(msg=False))
        print("\nMax percentage in stocks", n)
        print("State of the solution", LpStatus[prob.status])
        print(f"x1 (Stocks): {x1.varValue}")
        print(f"x2 (Bonds): {x2.varValue}")
        print(f"x3 (Savings): {x3.varValue}")
        print(f"Total max benefit: {value(prob.objective)}$")



Max percentage in stocks 0.2
State of the solution Optimal
x1 (Stocks): 20000.0
x2 (Bonds): 60000.0
x3 (Savings): 20000.0
Total max benefit: 5000.0$

Max percentage in stocks 0.3
State of the solution Optimal
x1 (Stocks): 30000.0
x2 (Bonds): 50000.0
x3 (Savings): 20000.0
Total max benefit: 5300.0$

Max percentage in stocks 0.4
State of the solution Optimal
x1 (Stocks): 40000.0
x2 (Bonds): 40000.0
x3 (Savings): 20000.0
Total max benefit: 5600.0$

Max percentage in stocks 0.5
State of the solution Optimal
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 5900.0$

Max percentage in stocks 0.6
State of the solution Optimal
x1 (Stocks): 60000.0
x2 (Bonds): 20000.0
x3 (Savings): 20000.0
Total max benefit: 6200.0$

Max percentage in stocks 0.7
State of the solution Optimal
x1 (Stocks): 70000.0
x2 (Bonds): 10000.0
x3 (Savings): 20000.0
Total max benefit: 6500.0$

Max percentage in stocks 0.8
State of the solution Optimal
x1 (Stocks): 80000.0
x2 (Bonds): 0.0
x3 


#### Minimum % in Savings (x3 ≥ n(x1 + x2 + x3))
As minimum savings increases (0% → 60%), more money moves to low-return savings and total return falls:

| Min % in Savings | Stocks | Bonds | Savings | Return |
|----|----|----|----|----|
| 0% | 50k | 50k | 0 | $6500 |
| 20% | 50k | 30k | 20k | $5900 |
| 60% | 40k | 0 | 60k | $4400 |

**Higher required savings = lower return.**

In [25]:
#SENSIBILITY ANALYSIS: x3 >= n*(x1+x2+x3)

for n in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    prob = LpProblem("Max_return_portfolios", LpMaximize)

    x1 = LpVariable("Stocks", lowBound = 0)
    x2 = LpVariable("Bonds", lowBound = 0)
    x3 = LpVariable("Savings", lowBound = 0)
    y = LpVariable("Return", lowBound = 0)

    prob += 0.08*x1 + 0.05*x2 + 0.02*x3, "Max benefit"
    prob += x1 + x2 + x3 <= 100000, "Total budget"
    prob += x1 <= 0.5*(x1+x2+x3), "Max 50 stocks"
    prob += x3 >= n*(x1+x2+x3), "Min n savings"
    prob += x1 >= 0, "x1 higher 0"
    prob += x2 >= 0, "x2 higher 0"
    prob += x3 >= 0, "x3 higher 0"

    if x3 >= n*(x1+x2+x3):
        prob.solve(PULP_CBC_CMD(msg=False))
        print("\nMin percentage in savings", n)
        print("State of the solution", LpStatus[prob.status])
        print(f"x1 (Stocks): {x1.varValue}")
        print(f"x2 (Bonds): {x2.varValue}")
        print(f"x3 (Savings): {x3.varValue}")
        print(f"Total max benefit: {value(prob.objective)}$")
        


Min percentage in savings 0
State of the solution Optimal
x1 (Stocks): 50000.0
x2 (Bonds): 50000.0
x3 (Savings): 0.0
Total max benefit: 6500.0$

Min percentage in savings 0.1
State of the solution Optimal
x1 (Stocks): 50000.0
x2 (Bonds): 40000.0
x3 (Savings): 10000.0
Total max benefit: 6200.0$

Min percentage in savings 0.2
State of the solution Optimal
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 5900.0$

Min percentage in savings 0.3
State of the solution Optimal
x1 (Stocks): 50000.0
x2 (Bonds): 20000.0
x3 (Savings): 30000.0
Total max benefit: 5600.0$

Min percentage in savings 0.4
State of the solution Optimal
x1 (Stocks): 50000.0
x2 (Bonds): 10000.0
x3 (Savings): 40000.0
Total max benefit: 5300.0$

Min percentage in savings 0.5
State of the solution Optimal
x1 (Stocks): 50000.0
x2 (Bonds): 0.0
x3 (Savings): 50000.0
Total max benefit: 5000.0$

Min percentage in savings 0.6
State of the solution Optimal
x1 (Stocks): 40000.0
x2 (Bonds): 0.0
x3 (Sa

### Effect of Changing Expected Returns

The sensitivity tests show that the portfolio reacts with **threshold (jump) behavior**: the model switches to the asset with the highest return once it clearly becomes the most profitable option.

#### Increasing the return on stocks
- When stock return is below ~5%, the model prefers bonds.
- Once the return reaches ≈ 5%, the portfolio shifts to include stocks.
- Higher stock returns gradually increase the total benefit.

**Insight:** Stocks only become attractive when their return matches or exceeds the bond return.

In [26]:
#SENSIBILITY ANALYSIS: n*x1 + 0.05*x2 + 0.02*x3

for n in [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]:
    prob = LpProblem("Max_return_portfolios", LpMaximize)

    x1 = LpVariable("Stocks", lowBound = 0)
    x2 = LpVariable("Bonds", lowBound = 0)
    x3 = LpVariable("Savings", lowBound = 0)
    y = LpVariable("Return", lowBound = 0)

    prob += n*x1 + 0.05*x2 + 0.02*x3, "Max benefit"
    prob += x1 + x2 + x3 <= 100000, "Total budget"
    prob += x1 <= 0.5*(x1+x2+x3), "Max 50 stocks"
    prob += x3 >= 0.2*(x1+x2+x3), "Min 20 savings"
    prob += x1 >= 0, "x1 higher 0"
    prob += x2 >= 0, "x2 higher 0"
    prob += x3 >= 0, "x3 higher 0"

    prob.solve(PULP_CBC_CMD(msg=False))
    print("\nState of the solution", LpStatus[prob.status])
    print("n*x1 + 0.05*x2 + 0.02*x3, n =",n)
    print(f"x1 (Stocks): {x1.varValue}")
    print(f"x2 (Bonds): {x2.varValue}")
    print(f"x3 (Savings): {x3.varValue}")
    print(f"Total max benefit: {value(prob.objective)}$")
        


State of the solution Optimal
n*x1 + 0.05*x2 + 0.02*x3, n = 0
x1 (Stocks): 0.0
x2 (Bonds): 80000.0
x3 (Savings): 20000.0
Total max benefit: 4400.0$



State of the solution Optimal
n*x1 + 0.05*x2 + 0.02*x3, n = 0.01
x1 (Stocks): 0.0
x2 (Bonds): 80000.0
x3 (Savings): 20000.0
Total max benefit: 4400.0$

State of the solution Optimal
n*x1 + 0.05*x2 + 0.02*x3, n = 0.02
x1 (Stocks): 0.0
x2 (Bonds): 80000.0
x3 (Savings): 20000.0
Total max benefit: 4400.0$

State of the solution Optimal
n*x1 + 0.05*x2 + 0.02*x3, n = 0.03
x1 (Stocks): 0.0
x2 (Bonds): 80000.0
x3 (Savings): 20000.0
Total max benefit: 4400.0$

State of the solution Optimal
n*x1 + 0.05*x2 + 0.02*x3, n = 0.04
x1 (Stocks): 0.0
x2 (Bonds): 80000.0
x3 (Savings): 20000.0
Total max benefit: 4400.0$

State of the solution Optimal
n*x1 + 0.05*x2 + 0.02*x3, n = 0.05
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 4400.0$

State of the solution Optimal
n*x1 + 0.05*x2 + 0.02*x3, n = 0.06
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 4900.0$

State of the solution Optimal
n*x1 + 0.05*x2 + 0.02*x3, n = 0.07
x1 (Stocks): 5


#### Increasing the return on bonds
- Bonds do not dominate until their return becomes very high (≈ 9% or more).
- Before that, the portfolio still allocates heavily to stocks.

**Insight:** Bonds need to outperform stocks **by a large margin** to replace them in the portfolio.

In [27]:
#SENSIBILITY ANALYSIS: 0.08*x1 + n*x2 + 0.02*x3

for n in [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]:
    prob = LpProblem("Max_return_portfolios", LpMaximize)

    x1 = LpVariable("Stocks", lowBound = 0)
    x2 = LpVariable("Bonds", lowBound = 0)
    x3 = LpVariable("Savings", lowBound = 0)
    y = LpVariable("Return", lowBound = 0)

    prob += 0.08*x1 + n*x2 + 0.02*x3, "Max benefit"
    prob += x1 + x2 + x3 <= 100000, "Total budget"
    prob += x1 <= 0.5*(x1+x2+x3), "Max 50 stocks"
    prob += x3 >= 0.2*(x1+x2+x3), "Min 20 savings"
    prob += x1 >= 0, "x1 higher 0"
    prob += x2 >= 0, "x2 higher 0"
    prob += x3 >= 0, "x3 higher 0"

    prob.solve(PULP_CBC_CMD(msg=False))
    print("\nState of the solution", LpStatus[prob.status])
    print("0.08*x1 + n*x2 + 0.02*x3, n =", n)
    print(f"x1 (Stocks): {x1.varValue}")
    print(f"x2 (Bonds): {x2.varValue}")
    print(f"x3 (Savings): {x3.varValue}")
    print(f"Total max benefit: {value(prob.objective)}$")
        


State of the solution Optimal
0.08*x1 + n*x2 + 0.02*x3, n = 0
x1 (Stocks): 50000.0
x2 (Bonds): 0.0
x3 (Savings): 50000.0
Total max benefit: 5000.0$

State of the solution Optimal
0.08*x1 + n*x2 + 0.02*x3, n = 0.01
x1 (Stocks): 50000.0
x2 (Bonds): 0.0
x3 (Savings): 50000.0
Total max benefit: 5000.0$

State of the solution Optimal
0.08*x1 + n*x2 + 0.02*x3, n = 0.02
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 5000.0$

State of the solution Optimal
0.08*x1 + n*x2 + 0.02*x3, n = 0.03
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 5300.0$

State of the solution Optimal
0.08*x1 + n*x2 + 0.02*x3, n = 0.04
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 5600.0$

State of the solution Optimal
0.08*x1 + n*x2 + 0.02*x3, n = 0.05
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 5900.0$

State of the solution Optimal
0.08*x1 + n*x2 + 0.02*x3, n = 0.06
x1 (Stock


#### Increasing the return on savings
- Savings remain marginal until their return rises above ≈ 6%.
- When savings return becomes extremely high (≈ 9–10%), the portfolio shifts almost entirely to savings.

**Insight:** Savings only become the preferred asset when their return is **significantly higher** than both stocks and bonds.


In [28]:
#SENSIBILITY ANALYSIS: 0.08*x1 + 0.05*x2 + n*x3

for n in [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]:
    prob = LpProblem("Max_return_portfolios", LpMaximize)

    x1 = LpVariable("Stocks", lowBound = 0)
    x2 = LpVariable("Bonds", lowBound = 0)
    x3 = LpVariable("Savings", lowBound = 0)
    y = LpVariable("Return", lowBound = 0)

    prob += 0.08*x1 + 0.05*x2 + n*x3, "Max benefit"
    prob += x1 + x2 + x3 <= 100000, "Total budget"
    prob += x1 <= 0.5*(x1+x2+x3), "Max 50 stocks"
    prob += x3 >= 0.2*(x1+x2+x3), "Min 20 savings"
    prob += x1 >= 0, "x1 higher 0"
    prob += x2 >= 0, "x2 higher 0"
    prob += x3 >= 0, "x3 higher 0"

    prob.solve(PULP_CBC_CMD(msg=False))
    print("\nState of the solution", LpStatus[prob.status])
    print("0.08*x1 + 0.05*x2 + n*x3, n =", n)
    print(f"x1 (Stocks): {x1.varValue}")
    print(f"x2 (Bonds): {x2.varValue}")
    print(f"x3 (Savings): {x3.varValue}")
    print(f"Total max benefit: {value(prob.objective)}$")
        


State of the solution Optimal
0.08*x1 + 0.05*x2 + n*x3, n = 0
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 5500.0$

State of the solution Optimal
0.08*x1 + 0.05*x2 + n*x3, n = 0.01
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 5700.0$

State of the solution Optimal
0.08*x1 + 0.05*x2 + n*x3, n = 0.02
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 5900.0$

State of the solution Optimal
0.08*x1 + 0.05*x2 + n*x3, n = 0.03
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 6100.0$

State of the solution Optimal
0.08*x1 + 0.05*x2 + n*x3, n = 0.04
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 6300.0$

State of the solution Optimal
0.08*x1 + 0.05*x2 + n*x3, n = 0.05
x1 (Stocks): 50000.0
x2 (Bonds): 30000.0
x3 (Savings): 20000.0
Total max benefit: 6500.0$

State of the solution Optimal
0.08*x1 + 0.05*x2 + n*x3, n = 0.06
x


**Summary:**  
Stocks drive the portfolio’s performance.  
Higher savings requirements or low stock caps reduce returns significantly.


## **SOLVING THE GENERAL EQUATION** ##
### **SIMPLEX METHOD** ###



### Step 1: Building the Initial Tableau

Here, we define all **symbolic variables** used in the problem:
- Decision variables: $ x_1, x_2, x_3 $
- Slack variables: $ s_1, s_2 $
- Cost coefficients: $ c_1, c_2, c_3, c_4, c_5 $
- Parameters $ \alpha $ and $ \beta $ are used to express the problem symbolically.

We know that $r_1$>$r_2$>$r_3$, and that $x_1<\alpha(x_1+x_2+x_3)$ and $x_3>\beta(x_1+x_2+x_3)$

The **initial tableau** represents the system of constraints and the objective function.  
It has the structure:

|  | c | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | RHS | RATIO |
|:--|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| (1) | 0 | 1 | 1 | 1 | 1 | 0 | 1 | 1 |
| (2) | 0 | $1 - \alpha$ | $-\alpha$ | $-\alpha$ | 0 | 1 | 0 | 0 |
| (3) | 0 | $\beta$ | $\beta$ | $\beta - 1$ | 0 | 0 | 0 | 0 |
| --- | --- | --- | --- | --- | --- | --- | --- | ---|
| (z) | | 0 | 0 | 0 | 0 | 0 |  | |
| (c-z) | | $r_1$ | $r_2$ | $r_3$ | 0 | 0 |  | |

This tableau is printed as a Sympy matrix.  
Then, the code computes **ratios** in the first pivot column to determine which constraint will leave the basis.


### Additional explanation:

- **Cost coefficients ($c_j$):**  
  Each $c_j$ is the objective-function coefficient for variable $x_j$. The slack variables $s_1, s_2$ have cost 0 because they do not affect the original objective.

- **Computing $z$ (current objective value):**  
  Multiply each basic variable's RHS value by its cost and add them:  

  $
  z = \sum r_i \cdot b_i
  $

- **Reduced costs ($c_j - z_j$):**  
  Shows how the objective changes if we increase a non-basic variable: 

  $
  c_j - z_j = c_j - \sum_{\text{basic}} c_i \cdot a_{ij}
  $  

  A negative value means increasing that variable will improve the objective (for maximization).

- **Ratio test (for leaving variable):**  
  For each positive value in the pivot column: 

  $
  \text{ratio} = \frac{\text{RHS}}{\text{pivot column entry}}
  $ 
  
  The smallest non-negative ratio picks the leaving variable.

- **Pivot selection:**  
  - **Entering variable:** most positive coeficient in the objective row
  - **Leaving variable:** smallest positive ratio


In [29]:
import sympy as sp

# === Define symbols ===
α, β = sp.symbols('alpha beta', real=True)
x1, x2, x3, s1, s2 = sp.symbols('x1 x2 x3 s1 s2')
c1, c2, c3, c4, c5 = sp.symbols('c1 c2 c3 c4 c5')
c_j = sp.Matrix([c1, c2, c3, c4, c5])
r_symbols = sp.symbols('r1 r2 r3', real=True)

# Initial tableau (last column = RHS)
tableau = sp.Matrix([
    [1, 1, 1, 1, 0, 1],
    [1-α, -α, -α, 0, 1, 0],
    [β, β, β-1, 0, 0, 0]
])

variables = [x1, x2, x3, s1, s2]

basic_vars = []
basic_rows = []
row_to_r = {}

def display_tableau(t):
    sp.pretty_print(t)

# === Print initial tableau ===
print("Initial tableau:")
display_tableau(tableau)

# --- Compute initial ratios (for first pivot decision) ---
first_pivot_col = 0  # for example, x1 enters first
rhs_idx = tableau.cols - 1
print(f"\nInitial ratios to decide first pivot (col {first_pivot_col+1}):")
for i in range(tableau.rows):
    denom = tableau[i, first_pivot_col]
    rhs = tableau[i, rhs_idx]
    if denom.is_zero:
        ratio_val = sp.oo
    else:
        ratio_val = sp.simplify(rhs / denom)
    print(f"  Row {i+1}: ratio = {rhs}/{denom} = {ratio_val}")


Initial tableau:
⎡  1    1     1    1  0  1⎤
⎢                         ⎥
⎢1 - α  -α   -α    0  1  0⎥
⎢                         ⎥
⎣  β    β   β - 1  0  0  0⎦

Initial ratios to decide first pivot (col 1):
  Row 1: ratio = 1/1 = 1
  Row 2: ratio = 0/1 - alpha = 0
  Row 3: ratio = 0/beta = 0


### Step 2: Pivot Operation Definition

This function `pivot_tableau()` performs the algebraic **pivoting** procedure of the Simplex Method:

1. **Select the pivot element**, intersection of the entering column and leaving row.  
2. **Normalize** the pivot row by dividing it by the pivot element.  
3. **Eliminate** the pivot column entries in all other rows using row operations.  
4. **Update** which variable is basic in that row.

This implementation keeps everything symbolic with **Sympy**, allowing $ \alpha $ and $ \beta $ to remain as parameters.  

After each pivot, the new tableau is printed to show the updated system of equations.


In [30]:
def pivot_tableau(tableau, pivot_row, pivot_col, var_name, next_pivot_col=None):
    pivot_elem = sp.simplify(tableau[pivot_row, pivot_col])

    # --- Show pivot info ---
    print(f"\n➡️ Next pivot: variable {var_name}, row {pivot_row+1}, col {pivot_col+1} (pivot element = {pivot_elem})")

    # --- Pivoting ---
    tableau[pivot_row, :] = sp.simplify(tableau[pivot_row, :] / pivot_elem)
    for i in range(tableau.rows):
        if i != pivot_row:
            factor = tableau[i, pivot_col]
            tableau[i, :] = sp.simplify(tableau[i, :] - factor * tableau[pivot_row, :])

    # --- Update r_i mapping ---
    var_to_r = {x1: r_symbols[0], x2: r_symbols[1], x3: r_symbols[2]}
    r_i = var_to_r[var_name]
    if pivot_row in row_to_r:
        idx = basic_rows.index(pivot_row)
        basic_vars[idx] = var_name
    else:
        basic_vars.append(var_name)
        basic_rows.append(pivot_row)
    row_to_r[pivot_row] = r_i

    # --- Tableau after pivot ---
    print("\nTableau after pivot:")
    display_tableau(tableau)

    # --- z_subj ---
    num_vars = tableau.cols - 1
    z_subj = sp.zeros(1, num_vars)
    for row, r_i in row_to_r.items():
        row_vals = [sp.simplify(tableau[row, j]) for j in range(num_vars)]
        z_subj += sp.Matrix([[sp.simplify(v * r_i) for v in row_vals]])
    z_subj = sp.simplify(z_subj)
    print("\nz_subj:")
    sp.pretty_print(z_subj)

    # --- c_j - z_subj ---
    c_values = {c1: r_symbols[0], c2: r_symbols[1], c3: r_symbols[2], c4: 0, c5: 0}
    c_subs = c_j.subs(c_values)
    cj_minus_zj = sp.Matrix([[sp.simplify(c_subs[i,0]-z_subj[0,i]) for i in range(z_subj.cols)]])
    print("\nc_j - z_subj:")
    sp.pretty_print(cj_minus_zj)

    # --- Ratios after pivot (for next pivot) ---
    if next_pivot_col is not None:
        rhs_idx = tableau.cols - 1
        print(f"\nRatios to decide next pivot (col {next_pivot_col+1}):")
        for i in range(tableau.rows):
            denom = tableau[i, next_pivot_col]
            rhs = tableau[i, rhs_idx]
            if denom.is_zero:
                ratio_val = sp.oo
            else:
                ratio_val = sp.simplify(rhs / denom)
            print(f"  Row {i+1}: ratio = {rhs}/{denom} = {ratio_val}")

    return tableau

### Step 3: First Pivot (Variable x₁ Enters)

We pivot on column corresponding to $ x_1 $, row 3 — making $ x_1 $ a **basic variable**.

This represents solving the system for $ x_1 $ as the entering variable and updating the tableau accordingly.

After the pivot, the new tableau is printed in symbolic form, showing how each constraint changes.

|  | c |$x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | RHS | RATIO |
|:--|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| (1) | 0 | 0 | 0 | $\frac{1}{\beta}$ | 1 | 0 | 1 | $\beta$ |
| (2) | 0 | 0 | –1 | $\frac{-\alpha - \beta + 1}{\beta}$ | 0 | 1 | 0 | 0 |
| ($x_1$) | $r_1$ |1 | 1 | $\frac{\beta - 1}{\beta}$ | 0 | 0 | 0 | 0 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| (z) | |$r_1$ | $r_1$ | $r_1 - \frac{r_1}{\beta}$ | 0 | 0 |  |
| (c-z) | | 0 | $-r_1 + r_2$ | $-r_1 + r_3 + \frac{r_1}{\beta}$ | 0 | 0 |  |

In [31]:

# ===== Apply pivots in desired sequence =====
tableau = pivot_tableau(tableau, pivot_row=2, pivot_col=0, var_name=x1, next_pivot_col=2)


➡️ Next pivot: variable x1, row 3, col 1 (pivot element = beta)

Tableau after pivot:
⎡           1              ⎤
⎢0  0       ─       1  0  1⎥
⎢           β              ⎥
⎢                          ⎥
⎢       -α - β + 1         ⎥
⎢0  -1  ──────────  0  1  0⎥
⎢           β              ⎥
⎢                          ⎥
⎢         β - 1            ⎥
⎢1  1     ─────     0  0  0⎥
⎣           β              ⎦

z_subj:
⎡             r₁      ⎤
⎢r₁  r₁  r₁ - ──  0  0⎥
⎣             β       ⎦

c_j - z_subj:
⎡                        r₁      ⎤
⎢0  -r₁ + r₂  -r₁ + r₃ + ──  0  0⎥
⎣                        β       ⎦

Ratios to decide next pivot (col 3):
  Row 1: ratio = 1/1/beta = beta
  Row 2: ratio = 0/(-alpha - beta + 1)/beta = 0
  Row 3: ratio = 0/(beta - 1)/beta = 0


### Step 4: Second Pivot (Variable x₃ Enters)

The second pivot occurs in the column for $ x_3 $ and the first row.  
This makes $ x_3 $ a basic variable and continues simplifying the tableau.

Each pivot brings us closer to the final reduced form where all basic variables have nonnegative coefficients in the RHS, and all reduced costs are nonnegative — signaling optimality.

|  | c | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | RHS | RATIO |
|:--|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| ($x_3$) | $r_3$ | 0 | 0 | 1 | $\beta$ | 0 | $\beta$ | $\infty$ |
| (2) | 0 | 0 | –1 | 0 | $\alpha+\beta-1$ | 1 | $\alpha+\beta-1$ | $\alpha + \beta - 1$ |
| ($x_1$) | $r_1$ | 1 | 1 | 0 | $1-\beta$ | 0 | $1-\beta$ | $1 - \beta$ |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| (z) | | $r_1$ | $r_1$ | $r_3$ | $\beta \cdot r_3 - r_1 \cdot (\beta-1)$ | 0 |  |
| (c-z) | | 0 | $-r_1 + r_2$ | 0 | $-\beta \cdot r_3 + (\beta-1) \cdot r_1$ | 0 |  |

In [32]:
tableau = pivot_tableau(tableau, pivot_row=0, pivot_col=2, var_name=x3, next_pivot_col=1)


➡️ Next pivot: variable x3, row 1, col 3 (pivot element = 1/beta)

Tableau after pivot:
⎡0  0   1      β      0      β    ⎤
⎢                                 ⎥
⎢0  -1  0  α + β - 1  1  α + β - 1⎥
⎢                                 ⎥
⎣1  1   0    1 - β    0    1 - β  ⎦

z_subj:
[r₁  r₁  r₃  β⋅r₃ - r₁⋅(β - 1)  0]

c_j - z_subj:
[0  -r₁ + r₂  0  -β⋅r₃ + r₁⋅(β - 1)  0]

Ratios to decide next pivot (col 2):
  Row 1: ratio = beta/0 = oo
  Row 2: ratio = alpha + beta - 1/-1 = -alpha - beta + 1
  Row 3: ratio = 1 - beta/1 = 1 - beta


### Step 5: Final Pivot (Variable x₂ Enters)

In this last symbolic pivot, $ x_2 $ enters the basis, and the tableau is updated once again.

After this step, we obtain the **final tableau**, from which we can directly read the optimal symbolic solution for all variables.

|  | c |$x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | RHS | RATIO |
|:--|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| ($x_3$) | $r_3$ | 0 | 0 | 1 | $\beta$ | 0 | $\beta$ |  |
| ($x_2$) | $r_2$ | 0 | 1 | 0 | $1-\alpha-\beta$ | -1 | $1-\alpha-\beta$ |  |
| ($x_1$) | $r_1$ | 1 | 0 | 0 | $\alpha$ | 1 | $\alpha$ |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| (z) |  | $r_1$ | $r_2$ | $r_3$ | $\alpha \cdot r_1 + \beta \cdot r_3 - r_2 \cdot (\alpha+\beta-1)$ | $r_1 - r_2$ |  |
| (c-z) |  | 0 | 0 | 0 | $-\alpha \cdot r_1 - \beta \cdot r_3 + r_2 \cdot (\alpha+\beta-1)$ | $-r_1 + r_2$ |  |


In [33]:
tableau = pivot_tableau(tableau, pivot_row=1, pivot_col=1, var_name=x2)  # last pivot


➡️ Next pivot: variable x2, row 2, col 2 (pivot element = -1)

Tableau after pivot:
⎡0  0  1      β       0       β     ⎤
⎢                                   ⎥
⎢0  1  0  -α - β + 1  -1  -α - β + 1⎥
⎢                                   ⎥
⎣1  0  0      α       1       α     ⎦

z_subj:
[r₁  r₂  r₃  α⋅r₁ + β⋅r₃ - r₂⋅(α + β - 1)  r₁ - r₂]

c_j - z_subj:
[0  0  0  -α⋅r₁ - β⋅r₃ + r₂⋅(α + β - 1)  -r₁ + r₂]


### Step 6: Extracting the Symbolic Solution

Finally, the code scans each column to determine which variables are **basic** (columns with a single 1 and zeros elsewhere).
Their values are taken from the **RHS column** of the tableau.

The resulting solution is displayed as:

$x_1 = \alpha, \quad x_3 = \beta, \quad x_2 = 1 - \alpha - \beta$


and the corresponding objective value $ Z $ is printed.

A check is performed to confirm the symbolic result matches the expected theoretical solution.


In [43]:

# ===== Final symbolic solution =====
solution = {}
rhs_idx = tableau.cols - 1
for j, var in enumerate(variables):
    col = [sp.simplify(tableau[i,j]) for i in range(tableau.rows)]
    if col.count(1)==1 and col.count(0)==tableau.rows-1:
        row_idx = col.index(1)
        solution[var] = sp.simplify(tableau[row_idx,rhs_idx])
    else:
        solution[var] = sp.Integer(0)

print("Final basic solution:")
for var, val in solution.items():
    print(f"{var} = {val}")

Z_val = sp.simplify(tableau[-1,-1])
print(f"\nZ = {Z_val}")

expected = {x1: α, x3: β, x2: 1-α-β}
print("--------------------------")
print("Check vs expected:")
for v, exp_val in expected.items():
    actual = sp.simplify(solution[v])
    print(f"{v}: actual = {actual}, expected = {exp_val}, equal? {sp.simplify(sp.Eq(actual,exp_val))}")


Final basic solution:
x1 = alpha
x2 = -alpha - beta + 1
x3 = beta
s1 = 0
s2 = 0

Z = alpha
--------------------------
Check vs expected:
x1: actual = alpha, expected = alpha, equal? True
x3: actual = beta, expected = beta, equal? True
x2: actual = -alpha - beta + 1, expected = -alpha - beta + 1, equal? True


### Interpretation of the Final Basic Solution

This solution shows the values of each variable in terms of the parameters **α (alpha)** and **β (beta)**.

- x1 = α
- x2 = −α − β + 1
- x3 = β
- s1 = 0
- s2 = 0

The objective function value is:

- Z = α

This means the solution depends on the chosen values of α and β.


### Consistency Check

We compare the calculated values with the expected ones:

| Variable | Actual Value            | Expected Value           | Match? |
|---------|-------------------------|--------------------------|--------|
| x1      | α                       | α                        | True |
| x2      | −α − β + 1              | −α − β + 1               | True |
| x3      | β                       | β                        | True |

All values match the expected results, so the solution is consistent.
